<a href="https://colab.research.google.com/github/LeslieZhoa/Simple-Lora/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 环境测试
%cd /content/
! nvidia-smi

In [ ]:
#@title 环境准备
!rm -rf Simple-Lora
!git clone https://github.com/LeslieZhoa/Simple-Lora.git
%cd Simple-Lora/
!pip install -r requirements.txt
!pip uninstall torchtext -y
!git lfs install
!wget https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_caption_capfilt_large.pth -P ./pretrained_models
%cd pretrained_models
!git clone https://huggingface.co/bert-base-uncased
!git clone https://huggingface.co/naonovn/chilloutmix_NiPrunedFp32Fix
%cd ..

In [ ]:
#@title 基础模型safetensor转换
!python process/convert_original_stable_diffusion_to_difdusers.py \
    --checkpoint_path ./pretrained_models/chilloutmix_NiPrunedFp32Fix/chilloutmix_NiPrunedFp32Fix.safetensors \
    --dump_path ./pretrained_models/chilloutmixNiPruned_Tw1O --from_safetensors

In [ ]:
#@title 选择训练lora的数据
from google.colab import files
import cv2
from google.colab.patches import cv2_imshow
import shutil
import os.path as osp
import os


uploaded_imgs_path = 'dataset/custom'
if not osp.exists(uploaded_imgs_path) : os.makedirs(uploaded_imgs_path)

def upload_img():
  uploaded_img = files.upload()
  uploadded_img_name = list(uploaded_img.keys())[0]

  !mv "{uploadded_img_name}" "{uploaded_imgs_path}"

  print(f"move file {uploadded_img_name} to {uploaded_imgs_path} ")
  return os.path.join(uploaded_imgs_path,uploadded_img_name)
up_mode = '\u4E0A\u4F20' #@param ['默认', '上传']
if up_mode == '默认':
  img_path = 'dataset/custom/11.jpeg'
else:
  !rm dataset/custom/*
  img_path = upload_img()
img = cv2.imread(img_path)
cv2_imshow(img)

# 获取图片文本信息

In [ ]:
!python process/run_caption.py --img_base ./dataset/custom

查看dataset/custom中的txt文本,并设置替换文本

In [ ]:
#@markdown ####**替换文本设置:**
ori_text = "an asian woman" #@param {type: 'string'}
new_text = "<lyf>" #@param {type: 'string'}
with open('1.txt','w') as f:
  f.write(ori_text)
with open('2.txt','w') as f:
  f.write(new_text)
!python process/change_txt.py --img_base ./dataset/custom --ori_txt "$(head -n 1 1.txt)" --new_txt "$(head -n 1 2.txt)"

In [ ]:
#@markdown ####**设置训练分辨率**
#@markdown 由于colab显存限制，如果使用512分辨率会内存溢出，所以这里使用384分辨率，但是效果会差很多
resolution = 384 #@param {type: 'integer'}
os.environ['resolution'] = str(resolution)
!python  train.py  --batch_size 1 --dist --print_interval 1 --train_text_encoder --save_interval 300 --eval 0 --resolution $resolution

展示训练的lora效果了

In [ ]:
#@markdown ####**输入文本设置:**
prompt = "<dlrb>,solo, long hair, black hair, choker, breasts, earrings, blue eyes, jewelry, lipstick, makeup, dark, bare shoulders, mountain, night, upper body, dress, large breasts, ((masterpiece))" #@param {type: 'string'}
with open('1.txt','w') as f:
  f.write(prompt)
!python inference.py \
    --mode 'lora' \
    --lora_path checkpoint/Lora/000-00000600.pth \
    --prompt  "$(head -n 1 1.txt)" \
    --outpath results/1.png \
    --num_images_per_prompt 1 \
    --width 384 --height 384
img = cv2.imread('results/1.png')
cv2_imshow(img)

In [ ]:
#@title 删除一些模型，存储不够拉！！！
!rm -rf pretrained_models/bert-base-uncased \
     checkpoint/Lora/000-00000000.pth checkpoint/Lora/000-00000300.pth

In [ ]:
#@title 准备inpaiting模型
%cd pretrained_models
!git clone https://huggingface.co/runwayml/stable-diffusion-inpainting
%cd ..
!wget https://github.com/LeslieZhoa/LVT/releases/download/v0.0/face_parsing.pt -P pretrained_models

展示inpaiting效果，如果结合controlnet(由于显存限制，暂不演示)就可以生成自己的虚拟偶像了

In [ ]:
#@markdown ####**输入文本设置:**
prompt = "green hair,short hair,curly hair, green hair,beach,seaside" #@param {type: 'string'}
with open('1.txt','w') as f:
  f.write(prompt)
!python inference.py \
    --mode 'inpait' \
    --inpait_path pretrained_models/stable-diffusion-inpainting \
    --mask_area all \
    --ref_img results/1.png \
    --prompt  "$(head -n 1 1.txt)" \
    --outpath results/2.png \
    --num_images_per_prompt 1
img = cv2.imread('results/2.png')
cv2_imshow(img)